In [1]:
import folium
import os, shutil
from glob import glob
#%matplotlib inline  
import json
import satsearch
import geopandas as gpd
import numpy as np
import pandas as pd
import warnings
from shapely.geometry import  Polygon
import rioxarray
import satstac

from data import cropy


In [2]:
harran={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}


In [3]:
datajson=json.dumps(harran)
area=gpd.read_file(datajson)


In [5]:
cropy.VectorProcessing.show_vector(target_area=area)

In [6]:
m,intersec_df=cropy.VectorProcessing.show_intersection(target_area=area,base_vector_path='../data/raw/boundries/sentinel_tr_tiles.shp')

In [7]:
m

In [8]:
intersec_df.head()

,name,folders,descriptio,altitude,alt_mode,time_begin,time_end,time_when,GID_0,NAME_0,geometry
0,37SDA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.87506 37.04125 0.00000, 39.1097..."
1,37SDB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.86147 37.94208 0.00000, 39.1110..."
2,37SEA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99978 37.04658 0.00000, 40.2344..."
3,37SEB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99977 37.94759 0.00000, 40.2493..."


In [9]:
import data.cropy as cpy

In [62]:
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-11-20/2018-12-01'
band_list=['B02','B08']
cloud_percentage=80

In [63]:
#burasu stact query olmali
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)

In [64]:
stac_result=stac_query.stac_result

In [60]:
sentinel_items=stac_query.find_sentinel_item()

In [61]:
sentinel_items

[S2A_37SDA_20180801_0_L2A,
 S2A_37SDB_20180801_0_L2A,
 S2A_37SEA_20180801_0_L2A,
 S2A_37SEB_20180801_0_L2A]

In [65]:
ll=stac_result.items()

In [66]:
len(ll)

2

In [67]:
df=stac_query.show_result_df(items_list=ll)
df.head()

,datetime,platform,constellation,instruments,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated,geometry,data_coverage
0,2018-11-24 08:19:49,sentinel-2b,sentinel-2,msi,10,0,32637,37,S,EA,0,S2B_MSIL2A_20181124T081249_N0211_R078_T37SEA_2...,100,68.89,True,2020-08-31T01:47:58.968Z,2020-08-31T01:47:58.968Z,"POLYGON ((40.21879 36.05054, 38.99979 36.05673...",NaN
1,2018-11-24 08:19:38,sentinel-2b,sentinel-2,msi,10,0,32637,37,S,DB,0,S2B_MSIL2A_20181124T081249_N0211_R078_T37SDB_2...,100,66.84,True,2020-09-26T06:19:22.984Z,2020-09-26T06:19:22.984Z,"POLYGON ((37.87638 36.95257, 37.86148 37.94207...",100.0


In [16]:
result_list=stac_query.show_result_list(sentinel_items_list=sentinel_items)

In [17]:
stac_query.show_result_map(items_list=sentinel_items,overview=True)

In [18]:
stac_query.download_image(item_list=sentinel_items[1:2],band_list=band_list[0:1])

NameError: name '_Stac__create_log_file' is not defined

In [20]:
subset_image=stac_query.download_subset_image(item_list=sentinel_items[0:1],band_list=band_list[0:1],aoi=harran)

In [21]:
type(subset_image)

list

In [22]:
subset_image[0]['B02']

<xarray.DataArray (band: 1, y: 3878, x: 3831)>
dask.array<getitem, shape=(1, 3878, 3831), dtype=float64, chunksize=(1, 1220, 3831), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 4.1e+06 4.1e+06 4.1e+06 ... 4.061e+06 4.061e+06
  * x            (x) float64 4.715e+05 4.715e+05 ... 5.097e+05 5.098e+05
  * band         (band) int32 1
    spatial_ref  int32 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [1]:
import requests

In [2]:
url='https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B01.tif'
incorrect_url='https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/EB/2020/7/S2A_37SEB_20200731_0_L2A/B02.tif'

In [3]:
resp = requests.get(f'http://cog-validate.radiant.earth/api/validate?url={url}')
resp.json()['status']

'success'

In [4]:
resp = requests.get(f'http://cog-validate.radiant.earth/api/validate?url={incorrect_url}')
resp.json()['status']

'failure'

In [65]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import datetime

In [11]:
api = SentinelAPI('orttak', 'a2k5i4f313', 'https://scihub.copernicus.eu/dhus')

In [31]:
 api.query?

Signature: api.query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords)
Docstring:
Query the OpenSearch API with the coordinates of an area, a date interval
and any other search keywords accepted by the API.

Parameters
----------
area : str, optional
    The area of interest formatted as a Well-Known Text string.
date : tuple of (str or datetime) or str, optional
    A time interval filter based on the Sensing Start Time of the products.
    Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
    The timestamps can be either a Python datetime or a string in one of the
    following formats:

        - yyyyMMdd
        - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
        - yyyy-MM-ddThh:mm:ssZ
        - NOW
        - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
        - NOW+<n>DAY(S)
        - yyyy-MM-ddThh:mm:ssZ-<n>DAY(S)
        - NOW/DAY (or HOUR, MONTH etc.) - rounds the value to the given unit

    Alternatively, an already

In [152]:
a.properties['sentinel:product_id']

'S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'

In [ ]:
a.

In [68]:
date = datetime.date(2020, 2, 20)
date += datetime.timedelta(days=1)
date

datetime.date(2020, 2, 21)

In [122]:
fr=a.date+datetime.timedelta(days=1)

In [123]:
fr

datetime.date(2018, 7, 13)

In [127]:
# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(a.geometry)
products = api.query(footprint,
                     #date = ('20181219', date(2018, 12, 29)),
                     date=(tt,fr),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 30))

In [142]:
products.get?

Signature: products.get(key, default=None, /)
Docstring: Return the value for key if key is in the dictionary, else default.
Type:      builtin_function_or_method


In [141]:
pp

<function odict_items.isdisjoint>

In [126]:
sat_df=api.to_geodataframe(products)
sat_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,ingestiondate,beginposition,endposition,...,granuleidentifier,datastripidentifier,highprobacloudspercentage,mediumprobacloudspercentage,notvegetatedpercentage,snowicepercentage,unclassifiedpercentage,vegetationpercentage,waterpercentage,geometry
1d8974bd-ae0d-45b3-8c50-cd51aebe9a9a,S2A_MSIL1C_20180712T080611_N0206_R078_T37SEB_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:42:23.906,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((38.99977 37.94759, 40.24935 37.94096..."
4c51954d-7f66-499e-9b50-819a9d2418f6,S2A_MSIL1C_20180712T080611_N0206_R078_T37SCB_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:36:31.294,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.34824 36.94479, 37.37955 37.05283..."
72cbb4f9-7145-4ff2-aab2-6d212c9da489,S2A_MSIL1C_20180712T080611_N0206_R078_T37SCV_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:32:09.706,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((36.83563 35.13541, 36.83564 35.13546..."
00cc5cae-16bb-4aca-b8b3-9a03646b4954,S2A_MSIL1C_20180712T080611_N0206_R078_T37SDV_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:22:41.374,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.88807 36.13974, 39.10849 36.14485..."
faf78f4e-9a6a-4800-b854-e897d645cf58,S2A_MSIL1C_20180712T080611_N0206_R078_T37SEA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:20:36.850,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((38.99978 37.04658, 40.23444 37.04016..."


In [145]:
res=sat_df.loc[sat_df['title']=='S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148']

In [150]:
res.index.values[0]

'7be30c50-31fc-48c4-ab45-fddea9be7877'

In [146]:
api.download_all(res.index,directory_path='.')

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\akif\.conda\envs\geo\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "C:\Users\akif\.conda\envs\geo\lib\site-packages\urllib3\connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "C:\Users\akif\.conda\envs\geo\lib\site-packages\urllib3\connectionpool.py", line 849, in _validate_conn
    conn.connect()
  File "C:\Users\akif\.conda\envs\geo\lib\site-packages\urllib3\connection.py", line 356, in connect
    ssl_context=context)
  File "C:\Users\akif\.conda\envs\geo\lib\site-packages\urllib3\util\ssl_.py", line 359, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\akif\.conda\envs\geo\lib\ssl.py", line 423, in wrap_socket
    session=session
  File "C:\Users\akif\.conda\envs\geo\lib\ssl.py", line 870, in _create
    self.do_handshake()
  File "C:\Users\akif\.conda\envs\geo\lib

StopIteration: 

In [151]:
api.download('7be30c50-31fc-48c4-ab45-fddea9be7877',directory_path='.')

Product 7be30c50-31fc-48c4-ab45-fddea9be7877 is not online. Triggering retrieval from long term archive.


{'id': '7be30c50-31fc-48c4-ab45-fddea9be7877',
 'title': 'S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148',
 'size': 1130256051,
 'md5': '886946BD039CC37EDEC270507615C9D9',
 'date': datetime.datetime(2018, 7, 12, 8, 6, 11, 24000),
 'footprint': 'POLYGON((37.875064737353405 37.04124978452211,39.10975932077869 37.04653228101576,39.1083684574531 36.05667498554276,37.88931628784403 36.05157871290146,37.875064737353405 37.04124978452211))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('7be30c50-31fc-48c4-ab45-fddea9be7877')/$value",
 'Online': False,
 'Creation Date': datetime.datetime(2018, 7, 12, 12, 30, 27, 534000),
 'Ingestion Date': datetime.datetime(2018, 7, 12, 12, 22, 39, 590000),
 'path': '.\\S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148.zip',
 'downloaded_bytes': 0}

In [74]:
api.get_product_odata('c59cdd69-3af0-48cb-9e7f-9bfde928d3cd')

{'id': 'c59cdd69-3af0-48cb-9e7f-9bfde928d3cd',
 'title': 'S2A_MSIL1C_20180712T080611_N0206_R078_T37SDA_20180712T101626',
 'size': 827057975,
 'md5': 'ED2207B7E2973F65BF097A0B2DF8704E',
 'date': datetime.datetime(2018, 7, 12, 8, 6, 11, 24000),
 'footprint': 'POLYGON((37.875064737353405 37.04124978452211,39.10975932077869 37.04653228101576,39.1083684574531 36.05667498554276,37.88931628784403 36.05157871290146,37.875064737353405 37.04124978452211))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('c59cdd69-3af0-48cb-9e7f-9bfde928d3cd')/$value",
 'Online': False,
 'Creation Date': datetime.datetime(2018, 7, 12, 23, 44, 57, 116000),
 'Ingestion Date': datetime.datetime(2018, 7, 12, 22, 0, 25, 355000)}

In [82]:
api.download('37b60075-6921-4e20-ba20-fbc1c8393e66',directory_path='.')

Downloading:   4%|██▎                                                             | 40.9M/1.16G [00:29<13:22, 1.39MB/s]


KeyboardInterrupt: 

In [28]:
api.get_product_odata('S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148')

SentinelAPIError: HTTP status 404 Not Found: InvalidKeyException : Invalid key (S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148) to access Products

""


In [180]:
sat_df

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,ingestiondate,beginposition,endposition,...,granuleidentifier,datastripidentifier,highprobacloudspercentage,mediumprobacloudspercentage,notvegetatedpercentage,snowicepercentage,unclassifiedpercentage,vegetationpercentage,waterpercentage,geometry
1d8974bd-ae0d-45b3-8c50-cd51aebe9a9a,S2A_MSIL1C_20180712T080611_N0206_R078_T37SEB_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:42:23.906,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((38.99977 37.94759, 40.24935 37.94096..."
4c51954d-7f66-499e-9b50-819a9d2418f6,S2A_MSIL1C_20180712T080611_N0206_R078_T37SCB_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:36:31.294,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.34824 36.94479, 37.37955 37.05283..."
72cbb4f9-7145-4ff2-aab2-6d212c9da489,S2A_MSIL1C_20180712T080611_N0206_R078_T37SCV_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:32:09.706,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((36.83563 35.13541, 36.83564 35.13546..."
00cc5cae-16bb-4aca-b8b3-9a03646b4954,S2A_MSIL1C_20180712T080611_N0206_R078_T37SDV_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:22:41.374,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((37.88807 36.13974, 39.10849 36.14485..."
faf78f4e-9a6a-4800-b854-e897d645cf58,S2A_MSIL1C_20180712T080611_N0206_R078_T37SEA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:20:36.850,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((38.99978 37.04658, 40.23444 37.04016..."
cea64d7e-9f9d-408e-ae6f-ea0189b1c0bb,S2A_MSIL1C_20180712T080611_N0206_R078_T37SEV_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2018-07-12T08:06:11.024Z, Instrument: MS...",false,2018-07-12 08:06:11.024,2018-07-12 22:18:16.174,2018-07-12 08:06:11.024,2018-07-12 08:06:11.024,...,S2A_OPER_MSI_L1C_TL_SGS__20180712T101626_A0159...,S2A_OPER_MSI_L1C_DS_SGS__20180712T101626_S2018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((40.20725 35.28445, 40.19473 35.24593..."
2ae65003-5dd5-4f96-b88f-49568d602f3e,S2A_MSIL1C_20180712T080611_N0206_R078_T37SCA_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/

In [177]:
sat_df.iloc[12] .title

'S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'

In [175]:
geo1.to_wkt()

'POLYGON ((37.8893272617132766 36.0515878295005834, 37.8750761132261715 37.0412408780802025, 39.1097480619874460 37.0465232766517971, 39.1083573665045066 36.0566840110034832, 37.8893272617132766 36.0515878295005834))'

In [118]:
sat_df.iloc[0].filename

'S2A_MSIL1C_20180712T080611_N0206_R078_T37SEB_20180712T101626.SAFE'

In [179]:
a='S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'
b='S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148'
a==b

True